In [ ]:
pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d jessicali9530/stanford-cars-dataset --unzip -p /content/stanford_cars


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import random
import cv2
import os
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input




In [ ]:
# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

# Select layers for feature extraction
layer_names = ['block1_conv2', 'block3_conv3', 'block5_conv3']
layer_outputs = [base_model.get_layer(layer).output for layer in layer_names]
feature_extractor = Model(inputs=base_model.input, outputs=layer_outputs)

# Load Stanford Car dataset images
car_images_path = "/content/stanford_cars/cars_test/cars_test"  # Change to "cars_train/cars_train" if needed
image_files = [f for f in os.listdir(car_images_path) if f.endswith('.jpg')]

# Ensure there are at least 5 images before sampling
if len(image_files) < 5:
    print(f"Warning: Only {len(image_files)} images found. Selecting all available images.")
    random_images = image_files  # Use all available images
else:
    random_images = random.sample(image_files, 5)  # Select 5 random images



In [ ]:
image_files = [f for f in os.listdir(car_images_path) if f.endswith('.jpg')]
random_images = random.sample(image_files, 5)  # Select 5 random images

In [ ]:

# Function to preprocess and extract features
def extract_and_visualize(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Extract features
    feature_maps = feature_extractor.predict(img_array)

    # Display original image
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 4, 1)
    plt.imshow(img)
    plt.title("Original Image")
    plt.axis('off')

    # Display feature maps
    for i, feature_map in enumerate(feature_maps):
        feature_map = np.squeeze(feature_map, axis=0)  # Remove batch dimension
        feature_map = np.mean(feature_map, axis=-1)  # Average over channels

        plt.subplot(1, 4, i+2)
        plt.imshow(feature_map, cmap='viridis')
        plt.title(f"{layer_names[i]} Output")
        plt.axis('off')

    plt.show()


In [ ]:

# Process 5 random images
for img_file in random_images:
    extract_and_visualize(os.path.join(car_images_path, img_file))

In [ ]:
!unzip "/content/Mask Wearing.v4-raw.yolov5pytorch.zip"


In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
!cd yolov5
!pip install -r requirements.txt  # install

In [ ]:
!python /content/yolov5/train.py --img 640 --batch 16 --epochs 50 --data /content/data.yaml --weights yolov5s.pt --device 0 --project runs/train --name mask_model --save-period 5


In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/train



In [ ]:
!python /content/yolov5/val.py --data /content/data.yaml --weights /content/runs/train/mask_model/weights/best.pt --img 640


In [ ]:
!python /content/yolov5/detect.py --weights /content/runs/train/mask_model/weights/best.pt --img 640 --source /content/test/images --conf 0.25 --iou 0.45


In [ ]:
!ls /content/test


In [ ]:
from IPython.display import Image, display
import glob

for image_path in glob.glob('/content/yolov5/runs/detect/exp2/*.jpg'):
    display(Image(filename=image_path))


In [ ]:
!python /content/yolov5/val.py --data /content/data.yaml --weights /content/runs/train/mask_model/weights/best.pt --task val --save-json --save-conf --name mask_model_val

In [ ]:
from IPython.display import Image
Image('/content/yolov5/runs/val/mask_model_val/confusion_matrix.png')
